In [10]:
def __return_waypoints_ind():
    d = np.array([
    [ 65, -36],
    [ 56, -21],
    [ 66, -46],
    [ 58, -48],
    [ 67, -29],
    [ 61, -16],
    [ 45, -32],
    [ 71, -43],
    # [ 80, -52],
    # [ 68, -58],
    [ 65, -54],
    [ 48, -20],
    [ 64, -21],
    [ 46, -14]])
    return d


def __return_waypoints_sdd():
    d = np.array([[24, 34],
       [16, 35],
       [24, 20],
       [14, 20],
       [14, 11],
       [ 6, 33],
       [ 7, 47],
       [23, 48]])
    return d

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import os, sys
from helpers.highlevel_sceneloader import HighLevelSceneLoader
from predictors.dataset_creator import TFDataSet
import tensorflow as tf
from predictors.dl_trainer import DLTrainer 
from predictors.extended_predictor import extended_predictor 
import matplotlib.pyplot as plt
import pickle
from helpers.graph import Graph
from helpers.accuracy_functions import destination_distance_l
import numpy as np
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')


''' set some parameters '''
# Model parameters
LSTM_LAYER_SIZE = 32
DENSE_LAYER_SIZE = 32
NUM_LSTM_LAYERS = 2
NUM_DENSE_LAYERS = 2
VARIABLE_INPUT_LENGTH = False

# Dataset
SEQ_IN_LEN = 3
SEQ_OUT_LEN = 4
NOISE_STD = .1
N_REPEATS = 1

BATCH_SIZE = 5
LENGTH_STRIDE = 2


# Training parameters
MAX_EPOCHS = 100
PATIENCE = 10

# For graph
GRAPH_DIST_THRESH = 4

''' get the data '''
ROOT = os.getcwd()

rel_p_img_b = 'helpers/analysed_vars_storage/img_bounds.xml'
rel_p_dests = 'helpers/analysed_vars_storage/destination_locations.xml'
p_img_bounds = os.path.join(ROOT, rel_p_img_b)
p_dest_locs = os.path.join(ROOT, rel_p_dests)

#TODO: older version of OpenTraj needed: "git checkout d249ba6951dd0f54b532fbe2ca6edc46b0d7093f"
opentraj_root = os.path.join(ROOT, 'OpenTraj')
root_datasets = os.path.join(ROOT, 'data/path_data')
sys.path.append(opentraj_root) # add package to pythonpath

scene_data = HighLevelSceneLoader(p_img_bounds, p_dest_locs)
# scene_data.load_ind(root_datasets, 7, 17)
scene_data.load_sdd(opentraj_root, "little", "video3")


''' create the graph instance '''    
interest_points = __return_waypoints_sdd()
g = Graph.from_matrices(interest_points, scene_data.destination_matrix, GRAPH_DIST_THRESH, .05)

df_signals = scene_data.df_to_lst_realxy_mats()
g.analyse_multiple_full_signals(df_signals, add_to_trams_mat=True)


<Element 'dataset' at 0x7f3b657dde00>
<Element 'dataset' at 0x7f3b657e84f0>


[(array(['w0', 'd6', 'w7'], dtype='<U2'),
  [(24, 34), (25.12452065, 45.36610442), (23, 48)]),
 ([], []),
 ([], []),
 (array(['d0', 'w5', 'w1', 'w0', 'd1'], dtype='<U2'),
  [(0.80188993, 33.06466289),
   (6, 33),
   (16, 35),
   (24, 34),
   (37.4084038, 33.74739443)]),
 (array(['d3', 'd4', 'w2', 'w0'], dtype='<U2'),
  [(18.44346834, 0.86828017), (23.60483923, 0.91962773), (24, 20), (24, 34)]),
 (array(['d7', 'w3', 'w4', 'd3'], dtype='<U2'),
  [(0.29354899, 22.89682134), (14, 20), (14, 11), (18.44346834, 0.86828017)]),
 (array(['d1', 'w0', 'd6', 'w7'], dtype='<U2'),
  [(37.4084038, 33.74739443), (24, 34), (25.12452065, 45.36610442), (23, 48)]),
 (array(['w0', 'w2'], dtype='<U2'), [(24, 34), (24, 20)]),
 (array(['d4', 'w2', 'w0', 'd1'], dtype='<U2'),
  [(23.60483923, 0.91962773), (24, 20), (24, 34), (37.4084038, 33.74739443)]),
 (array(['d2', 'w7', 'd6', 'w0', 'w2', 'd4'], dtype='<U2'),
  [(25.14758057, 53.29443775),
   (23, 48),
   (25.12452065, 45.36610442),
   (24, 34),
   (24, 20),


In [12]:

''' time to create df datasets '''
extra_features_dict = {
    "all_points": None,
    "all_destinations": None,
    "n_destinations": 5,
    "n_points": 5,
    "n_connected_points_after" : 3
}

# # Load data in order to not need to do calculations again
with open("data/pickle/ds_creation_d/ds_7to17_inputLabels3_4_low_noise.pickle", 'rb') as handle: #"data/pickle/ds_creation_d/bs1.pickle"
    my_ds_creation_dict = pickle.load(handle)

my_ds = TFDataSet.init_as_fixed_length(scene_data.traj_dataframe, graph=g, var_in_len=VARIABLE_INPUT_LENGTH, length_stride=LENGTH_STRIDE,
scale_list=["pos_x", "pos_y"], seq_in_length=SEQ_IN_LEN, label_length=SEQ_OUT_LEN,
extra_features_dict=extra_features_dict, noise_std=NOISE_STD, 
n_repeats=N_REPEATS, batch_size=BATCH_SIZE, ds_creation_dict=my_ds_creation_dict) #save_folder = "data/pickle/ds_creation_d/ds_7to17_inputLabels3_4_low_noise.pickle"


In [14]:
''' time for some model training '''

my_trainer = DLTrainer(max_epochs=MAX_EPOCHS, patience=PATIENCE)
my_trainer.LSTM_one_shot_predictor_named_i(my_ds, LSTM_LAYER_SIZE, DENSE_LAYER_SIZE, 
NUM_LSTM_LAYERS, NUM_DENSE_LAYERS, extra_features=['all_destinations'], 
var_time_len=VARIABLE_INPUT_LENGTH, size_dict=my_ds.size_dict)

from datetime import datetime 

folder_path = "data/model_weights/checkpoints/%s" % (datetime.now().strftime("%d_%m_%Y__%H_%M"))
save_path = os.path.join(folder_path, "model_3_4_w_dests_sdd.pickle")            
if not os.path.exists(folder_path):
    os.mkdir(folder_path)
my_trainer.compile_and_fit2(my_ds.tf_ds_dict["train"], my_ds.tf_ds_dict["val"], save_path=save_path)

3
KerasTensor(type_spec=TensorSpec(shape=(None, 3, 2), dtype=tf.float32, name='in_xy'), name='in_xy', description="created by layer 'in_xy'")
KerasTensor(type_spec=TensorSpec(shape=(None, 32), dtype=tf.float32, name=None), name='flatten_1/Reshape:0', description="created by layer 'flatten_1'")
KerasTensor(type_spec=TensorSpec(shape=(None, 9, 3), dtype=tf.float32, name='all_destinations'), name='all_destinations', description="created by layer 'all_destinations'")
Epoch 1/100
1862/1862 [==============================] - 27s 10ms/step - loss: 0.0554 - mean_absolute_error: 0.1536 - val_loss: 0.0471 - val_mean_absolute_error: 0.1605

Epoch 00001: val_loss improved from inf to 0.04714, saving model to data/model_weights/checkpoints/11_10_2021__23_02/model_3_4_w_dests_sdd.pickle
Alternative save happened on for epoch 0 happened on data/model_weights/checkpoints/11_10_2021__23_02/model_3_4_w_dests_sdd.pickle for val_loss of 0.0471.
Epoch 2/100
1862/1862 [==============================] - 20s 

In [9]:
# Load the 2 models (with and without destination included as a feature)

In [17]:
''' or some model loading '''

# first with destinations
model_path = "data/model_weights/checkpoints/11_10_2021__23_02/model_3_4_w_dests_sdd.pickle"

my_trainer_w_dest = DLTrainer(max_epochs=MAX_EPOCHS, patience=PATIENCE)
my_trainer_w_dest.LSTM_one_shot_predictor_named_i(my_ds, 32, 32, 
NUM_LSTM_LAYERS, NUM_DENSE_LAYERS, extra_features=["all_destinations"], 
var_time_len=VARIABLE_INPUT_LENGTH, size_dict=my_ds.size_dict, epistemic=False, dropout_rate=0.05)
my_trainer_w_dest.load_weights(model_path)

# and also one without destinations
model_path = "data/model_weights/checkpoints/11_10_2021__22_22/model_3_4_wo_dests_sdd.pickle"

my_trainer_wo_dest = DLTrainer(max_epochs=MAX_EPOCHS, patience=PATIENCE)
my_trainer_wo_dest.LSTM_one_shot_predictor_named_i(my_ds, 32, 32, 
NUM_LSTM_LAYERS, NUM_DENSE_LAYERS, extra_features=[], 
var_time_len=VARIABLE_INPUT_LENGTH, size_dict=my_ds.size_dict, epistemic=False, dropout_rate=0.05)
my_trainer_wo_dest.load_weights(model_path)

3
KerasTensor(type_spec=TensorSpec(shape=(None, 3, 2), dtype=tf.float32, name='in_xy'), name='in_xy', description="created by layer 'in_xy'")
KerasTensor(type_spec=TensorSpec(shape=(None, 32), dtype=tf.float32, name=None), name='flatten_7/Reshape:0', description="created by layer 'flatten_7'")
KerasTensor(type_spec=TensorSpec(shape=(None, 9, 3), dtype=tf.float32, name='all_destinations'), name='all_destinations', description="created by layer 'all_destinations'")
3
KerasTensor(type_spec=TensorSpec(shape=(None, 3, 2), dtype=tf.float32, name='in_xy'), name='in_xy', description="created by layer 'in_xy'")
KerasTensor(type_spec=TensorSpec(shape=(None, 32), dtype=tf.float32, name=None), name='flatten_9/Reshape:0', description="created by layer 'flatten_9'")


In [18]:
''' Calculate KPIs for the whole test data set '''
# set up values where 
num_predicted_batches = 0.
total_w_dest = [0.] * len(g.destination_names)
total_wo_dest = [0.] * len(g.destination_names)

test_ds = my_ds.tf_ds_dict["test"]

for batch in tqdm(iter(test_ds)):
    batch_in = batch[0]
    batch_out = batch[1]

    # predict without destination
    pred_wo_dest = my_trainer_wo_dest.predict_repetitively_dict(batch_in, False, 25, False)

    for dest_name in g.destination_names:
        dest_loc = g.dest_dict[dest_name]

        # get one hot vector to dest
        dest_index = g.destinations_indices_dict[dest_name]
        min_dest_index = min(g.destinations_indices_dict.values())

        new_probs = tf.one_hot(
            [dest_index-min_dest_index], g.num_destinations, on_value=None, off_value=None, axis=None, dtype=tf.float32, name=None
        )
        new_probs = tf.reshape(new_probs, [-1, 1])

        in_shape = batch_in["in_xy"].shape
        if len(in_shape)==2:
            batch_size = 1
        else: 
            batch_size = in_shape[-3]
        probs_lst = [new_probs for i in range(batch_size)]
        new_probs=tf.stack(probs_lst)

        # implement the one hot vector
        input_dict_c = dict(batch_in)
        dest_loc_one_hot = tf.concat([input_dict_c["all_destinations"][:,:,0:2], new_probs], axis=-1)
        input_dict_c["all_destinations"] = dest_loc_one_hot

        # print(dest_index)
        # print(dest_loc_one_hot)

        # do prediction with destination
        pred_w_dest = my_trainer_w_dest.predict_repetitively_dict(input_dict_c, False, 27, False)
        
        # now do kpi calculation
        kpi_w_dest = destination_distance_l(pred_w_dest, dest_loc, only_endpoint=False)
        kpi_wo_dest = destination_distance_l(pred_wo_dest, dest_loc, only_endpoint=False)
        
        # print(pred_w_dest[:, :-1, :])
        # print(pred_wo_dest[:, :-1, :])
        # print("With destination: %s"%np.average(kpi_w_dest))
        # print("Without destination: %s"%np.average(kpi_wo_dest))

        total_w_dest[dest_index-min_dest_index] += np.sum(kpi_w_dest) / np.max(kpi_w_dest.shape)
        total_wo_dest[dest_index-min_dest_index] += np.sum(kpi_wo_dest) / np.max(kpi_w_dest.shape)

    num_predicted_batches += 1

    # if num_predicted_batches > 20:
    #     break

print("With destination: %s"%[el/num_predicted_batches for el in total_w_dest])
print("Without destination: %s"%[el/num_predicted_batches for el in total_wo_dest])


    
    # print("With destination: %s"%(total_w_dest/num_predictions))
    # print("Without destination: %s"%(total_wo_dest/num_predictions))


547it [20:32,  2.25s/it]

With destination: [17.853980063093395, 17.152611579018213, 25.373647371730687, 16.74323386948167, 19.45001740837194, 15.610221554363267, 13.456024060632982, 21.87390619021967, 22.17893098503176]
Without destination: [19.419836652833276, 18.02128269776322, 20.397181625435397, 22.875803758629104, 21.411840887570733, 18.045361897161893, 15.10982915499526, 21.98991947654668, 17.965376838313777]


In [30]:
''' time for some model predictions '''
my_predictor_w_dest = extended_predictor(g, my_trainer_w_dest, 1)

nxt_unsc, nxt_sc = my_ds.example_dict("test", "in_xy")

# Let's extract just one path to make visualisation clearer
unscaled_ex = dict(nxt_unsc[0]), nxt_unsc[1]
scaled_ex = dict(nxt_sc[0]), nxt_sc[1]
# let's get an example of length one
for key in unscaled_ex[0].keys():
    unscaled_ex[0][key] = tf.expand_dims(nxt_unsc[0][key][0], axis=0)
for key in scaled_ex[0].keys():
    scaled_ex[0][key] = tf.expand_dims(nxt_sc[0][key][0], axis=0)   

In [31]:
# Basic prediction, but repeated (one at a time)
#PROBLEM: input is scaled
assembled_output, destination_list, dest_prob_dict = my_predictor_w_dest.predict_to_destinations(input_dict=unscaled_ex[0], 
num_steps = 20, variable_len_input=VARIABLE_INPUT_LENGTH, num_predictions=15, norm_probs=True, abs_probs=False)
# Epistemic uncertainty prediction


# Find how many paths are within range of 4 meters